In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping

import argparse
import time
import random
import cv2
import numpy as np
import keras
import glob
from tqdm import tqdm

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D,UpSampling2D,concatenate
from keras.layers import BatchNormalization, ReLU, Activation, Concatenate, Conv2DTranspose, Reshape,Softmax
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.models import Model

from sklearn.utils import class_weight
random.seed(165)
import os, sys
os.environ['CUDA_VISIBLE_DEVICES']= '3'

In [ ]:
DATASET_PATH ="/home/azka/KLProjectCombined/Knee/KneeXrayData/ClsKLData/kneeKL224"
IMSIZE =128, 128


def normalize(img):
    mat = np.zeros(img.shape, dtype=np.int16)
    h, w = img.shape[:2]
    m1 = img.min()
    m2 = img.max()
    for j in range(h):
        for i in range(w):
            mat[j, i] = ((img[j, i] - m1) / (m2 - m1)) * 255
    return mat

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def find_joint(img):
    cols = img.sum(axis = 1)
    cols = np.array(cols,dtype='int64')
    slope = []
    for i in range(100, len(cols)-1):
        slope.append(float(cols[i+1] - cols[i]))
    j = np.argmin(slope) + 100
    return j

def Class5Label(lab): 

    lb = [0] *5
    lb[int(lab)] = 1
    return lb


def Class2Label(label): 
    if label=='2' or label=='1' or label=='0':
        lab='0'
    elif label=='3' or label=='4':
        lab='1'

    lb = [0] *2
    lb[int(lab)] = 1
    return lb

def DataLoad(imdir):
    folders = glob.glob(os.path.join(imdir,'*'))
    img_a = []
    img_b = []
    img_c = []
    lb = []
    lb=[]
    cls = []
    lr = []
    lb5=[]
    for folder in folders:
        impath = glob.glob(os.path.join(folder,'*.png'))
        label = folder[-1]
        print('Loading', len(impath), label, 'images ...', 'HOHO')
        for i, p in enumerate(impath):
            img_whole = cv2.imread(p,0)
            h, w = img_whole.shape
            if impath[0][-5] == 'L':
                im_a = img_whole
                img_a.append(im_a)
                img_b.append(im_a)
                lb.append(Class2Label(label))
                cls.append(int(label))
                lr.append([1,0])
                lb5.append(Class5Label(label))

            elif impath[0][-5] == 'R':
                im_b = img_whole
                img_whole = cv2.flip(img_whole, 1)
                im_a = img_whole
                img_a.append(im_a)
                img_b.append(im_b)
                #img_c.append(im_c)
                lb.append(Class2Label(label))
                cls.append(int(label))
                lr.append([0,1])
                lb5.append(Class5Label(label))
    return img_a, img_b, img_c, lb, np.array(cls), lr, lb5


def ImagePreprocessing(img):
    h, w = IMSIZE
    print('Preprocessing ...')
    for i, im, in tqdm(enumerate(img)):
        tmp = cv2.resize(im, dsize=(w, h), interpolation=cv2.INTER_AREA)
        tmp = NormalizeData(cv2.equalizeHist(tmp))
        img[i] = tmp
    print(len(img), 'images processed!')
    return img

imageA, imageB, _, labels,classes, lr,lb5 = DataLoad(os.path.join(DATASET_PATH, 'train'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

imageB = ImagePreprocessing(imageB)
imageB = np.array(imageB)
imageB = np.expand_dims(imageB, axis=-1)

lb5=np.array(lb5)
labels = np.array(labels)
lr = np.array(lr)

X_train = imageA
X_trainLR = imageB
Y_train = labels
Y_train2=lb5
Y_trainLR=lr

imageA, imageB, _, labels,classes, lr,lb5 = DataLoad(os.path.join(DATASET_PATH, 'val'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

imageB = ImagePreprocessing(imageB)
imageB = np.array(imageB)
imageB = np.expand_dims(imageB, axis=-1)

labels = np.array(labels)
lr = np.array(lr)
lb5=np.array(lb5)

#labelsL = np.array(labelsL)
#labelsR = np.array(labelsR)
lr = np.array(lr)
X_val = imageA
X_valLR = imageB
Y_val = labels
Y_val2=lb5
Y_valLR=lr

In [ ]:
imageA, imageB, _, labels,classes, lr,lb5 = DataLoad(os.path.join(DATASET_PATH, 'test'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

imageB = ImagePreprocessing(imageB)
imageB = np.array(imageB)
imageB = np.expand_dims(imageB, axis=-1)

labels = np.array(labels)
lr = np.array(lr)
lb5=np.array(lb5)

X_test = imageA

X_testLR = imageB
Y_test = labels
Y_test2=lb5
Y_testLR=lr

# Data for sub-classifiers

In [ ]:

def Class2Label(label): 
    if label=='3':
        lab='0'
    elif label=='4':
        lab='1'

    lb = [0] *2
    lb[int(lab)] = 1
    return lb

IMSIZE=128,128
def DataLoad34(imdir):
    folders = glob.glob(os.path.join(imdir,'*'))
    img_a = []
    img_b = []
    img_c = []
    lb = []
    lb=[]
    cls = []
    lr = []
    lb5=[]
    for folder in folders:
        impath = glob.glob(os.path.join(folder,'*.png'))
        label = folder[-1]
        if label=='3' or label=='4':
            print('Loading', len(impath), label, 'images ...', 'HOHO')
            for i, p in enumerate(impath):
                img_whole = cv2.imread(p,0)
                h, w = img_whole.shape
                if impath[0][-5] == 'L':
                    im_a = img_whole
                    #im_b = img_whole
                    #im_c = img_whole
                    img_a.append(im_a)
                    #img_b.append(im_b)
                    #img_c.append(im_c)
                    lb.append(Class2Label(label))
                    cls.append(int(label))
                    lr.append(0)
                    lb5.append(Class5Label(label))

                elif impath[0][-5] == 'R':
                    img_whole = cv2.flip(img_whole, 1)
                    im_a = img_whole
                    im_b = img_whole
                    #im_c = img_whole
                    img_a.append(im_a)
                    img_b.append(im_b)
                    #img_c.append(im_c)
                    lb.append(Class2Label(label))
                    cls.append(int(label))
                    lr.append(1)
                    lb5.append(Class5Label(label))
    return img_a, img_b, img_c, lb, np.array(cls), lr, lb5




imageA, imageB, _, labels,classes, lr,_ = DataLoad34(os.path.join(DATASET_PATH, 'train'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)
labels = np.array(labels)
lr = np.array(lr)

X_train34 = imageA
Y_train34 = labels
imageA, imageB, _, labels,classes, lr,_ = DataLoad34(os.path.join(DATASET_PATH, 'val'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

labels = np.array(labels)
lr = np.array(lr)
lb5=np.array(lb5)
#labelsL = np.array(labelsL)
#labelsR = np.array(labelsR)
lr = np.array(lr)
X_val34 = imageA
Y_val34 = labels


imageA, imageB, _, labels,classes, lr,_ = DataLoad34(os.path.join(DATASET_PATH, 'test'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

labels = np.array(labels)
lr = np.array(lr)

lb5=np.array(lb5)

X_test34 = imageA
Y_test34 = labels



In [ ]:
def Class2Label(lab): 
    lb = [0] *3
    lb[int(lab)] = 1
    return lb

#IMSIZE=512,512

def DataLoad012(imdir):
    folders = glob.glob(os.path.join(imdir,'*'))
    img_a = []
    img_b = []
    img_c = []
    lb = []
    lb=[]
    cls = []
    lr = []
    lb5=[]
    for folder in folders:
        impath = glob.glob(os.path.join(folder,'*.png'))
        label = folder[-1]
        if label=='0' or label=='1' or label=='2':
            print('Loading', len(impath), label, 'images ...', 'HOHO')
            for i, p in enumerate(impath):
                img_whole = cv2.imread(p,0)
                h, w = img_whole.shape
                if impath[0][-5] == 'L':
                    im_a = img_whole
                    img_a.append(im_a)
                    lb.append(Class2Label(label))
                    cls.append(int(label))
                    lr.append(0)

                elif impath[0][-5] == 'R':
                    im_b = img_whole
                    img_whole = cv2.flip(img_whole, 1)
                    im_a = img_whole
                    img_a.append(im_a)
                    
                    lb.append(Class2Label(label))
                    cls.append(int(label))
                    lr.append(1)
    return img_a, lb, np.array(cls), lr, lb5




imageA,labels,classes, lr,_ = DataLoad012(os.path.join(DATASET_PATH, 'train'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)
labels = np.array(labels)
lr = np.array(lr)

X_train012 = imageA
Y_train012 = labels


imageA, labels,classes, lr,_ = DataLoad012(os.path.join(DATASET_PATH, 'val'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

labels = np.array(labels)
lr = np.array(lr)
lb5=np.array(lb5)
lr = np.array(lr)
X_val012 = imageA
Y_val012 = labels


imageA,labels,classes, lr,_ = DataLoad012(os.path.join(DATASET_PATH, 'test'))
imageA = ImagePreprocessing(imageA)
imageA = np.array(imageA)
imageA = np.expand_dims(imageA, axis=-1)

labels = np.array(labels)
lr = np.array(lr)

X_test012 = imageA
Y_test012 = labels


# My encoder

In [ ]:
def encoder(input_img):    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv4 = BatchNormalization(name='encoder')(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='decoder')(up2) # 28 x 28 x 1
    return decoded

# Using Latent rep from Encoder

In [ ]:
def fc(x):
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(512, (3, 3), padding='same')(x)

    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    flatten=Flatten()(x)
    x = Dense(1000,activation='relu')(flatten)# for multiclass classifier i used this
    x = Dense(512,activation='relu')(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(2)(x)
    x = Softmax(name='classifier')(x)
    return x

# Training Binary classifier

In [ ]:
def get_autoencoder():
    input_img = Input(shape=(128,128,1),name="input")
    encode = encoder(input_img)
    decode=decoder(encode)
    classifier=fc(encode)
    
    model = Model(inputs=[input_img],outputs=[classifier,decode])
    return model

In [ ]:
autoNet = get_autoencoder()
autoNet.summary()

In [ ]:
def macro_f1(y, y_hat):
    bce=tf.keras.losses.BinaryCrossentropy()
    bce=bce(y,y_hat)
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
    
    specificity = tn / (tn + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    return 0.5*bce+1-(0.8*recall+0.2*specificity)

In [ ]:
from keras.optimizers import SGD 
optimizer = keras.optimizers.Adam(lr=0.0001)
sgd = SGD(0.0005, momentum=0.9, nesterov=False)
m = tf.keras.metrics.FalseNegatives()
acc = tf.keras.metrics.Accuracy()
logs_base_dir = "./logs"

tensorboard_callback = tf.keras.callbacks.TensorBoard(logs_base_dir, histogram_freq=1)
bce=tf.keras.losses.BinaryCrossentropy()
autoNet.compile(optimizer=optimizer,loss=[macro_f1,'mse'],loss_weights=[1,0.3],metrics=['accuracy'])
checkpoint_filepath='/home/azka/My KL Grade Project/Notebooks/modelmain'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_classifier_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('base_classifier.csv', separator=';')

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
sns.set(font_scale=2)
pred=autoNet.predict([X_test])
pred = pred[0].argmax(axis=1)
actual = Y_test.argmax(axis=1)
accuracy = (pred == actual).sum()/len(pred)
cm=confusion_matrix(actual,pred)
sns.heatmap(cm, annot=True, fmt='g')

# Training binary classifier for grade 3 and 4

In [ ]:
def get_subclassifier():
    input_img = Input(shape=(128,128,1),name="input")
    encode = encoder(input_img)
    decode=decoder(encode)
    classifier=fc(encode)
    model = Model(inputs=[input_img],outputs=[classifier,decode])
    return model

In [ ]:
binClassifier = get_subclassifier()
binClassifier.summary()

In [ ]:
for l1,l2 in zip(binClassifier.layers[:17],autoNet.layers[0:17]):
    l1.set_weights(l2.get_weights())

In [ ]:
for layer in binClassifier.layers[0:17]:
    layer.trainable = False

In [ ]:
from keras.optimizers import SGD 
optimizer = keras.optimizers.Adam(lr=0.0001)
sgd = SGD(0.0005, momentum=0.9, nesterov=False)
m = tf.keras.metrics.FalseNegatives()
acc = tf.keras.metrics.Accuracy()
bce=tf.keras.losses.BinaryCrossentropy()
binClassifier.compile(optimizer=optimizer,loss=[macro_f1,'mse'],loss_weights=[1,0.3],metrics=['accuracy'])
checkpoint_filepath='/home/azka/My KL Grade Project/Notebooks/model'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_classifier_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('bin_classifier.csv', separator=';')

In [ ]:
classify_train = binClassifier.fit([X_train34],[Y_train34,X_train34], batch_size=8,validation_data=([X_val34],[Y_val34,X_val34]),epochs=70,verbose=1,shuffle=True,callbacks=[csv_logger,model_checkpoint_callback])

In [ ]:
import pandas as pd
binClassifier.load_weights(checkpoint_filepath)
classify_train = pd.read_csv('bin_classifier.csv', sep=';')

In [ ]:
binClassifier.evaluate([X_train34],[Y_train34,X_train34])

In [ ]:
binClassifier.evaluate([X_val34],[Y_val34,X_val34])

In [ ]:
binClassifier.evaluate([X_test34],[Y_test34,X_test34])

In [ ]:
plt.plot(classify_train['classifier_accuracy'])
plt.plot(classify_train['val_classifier_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')

In [ ]:
plt.plot(classify_train['classifier_loss'])
plt.plot(classify_train['val_classifier_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
sns.set(font_scale=2)
pred=binClassifier.predict(X_test34)
pred = pred[0].argmax(axis=1)
actual = Y_test34.argmax(axis=1)
accuracy = (pred == actual).sum()/len(pred)
cm=confusion_matrix(actual,pred)
sns.heatmap(cm, annot=True, fmt='g')

# Grade 0,1,2 classifier

In [ ]:
from sklearn.utils import shuffle

X_test012, Y_test012 = shuffle(X_test012, Y_test012)
X_val012, Y_val012 = shuffle(X_val012, Y_val012)
X_train012, Y_train012 = shuffle(X_train012, Y_train012)

In [ ]:
def fc1(x):
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x=Dropout(0.3)(x)
    x= Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    
    x=Dropout(0.3)(x)
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size = (2,2), padding = 'same')(x) 
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x= Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    x=Flatten()(x)
    x = Dense(1000,activation='relu')(x)# for multiclass classifier i used this
    x = Dense(512,activation='relu')(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(3)(x)
    x = Softmax(name='classifier')(x)
    return x

In [ ]:
def get_subclassifier2():
    input_img = Input(shape=(128,128,1),name="input")
    encode = encoder(input_img)
    decode=decoder(encode)
    classifier=fc1(encode)
    model = Model(inputs=[input_img],outputs=[classifier,decode])
    return model

In [ ]:
mulClassifier = get_subclassifier2()
mulClassifier.summary()

In [ ]:
for l1,l2 in zip(mulClassifier.layers[:17],autoNet.layers[0:17]):
    l1.set_weights(l2.get_weights())

In [ ]:
for layer in mulClassifier.layers[0:17]:
    layer.trainable = False

In [ ]:
random.seed(45)

In [ ]:
from keras.optimizers import SGD 
import shutil
optimizer = keras.optimizers.Adam(lr=0.0001)
m = tf.keras.metrics.FalseNegatives()
acc = tf.keras.metrics.Accuracy()
logs_base_dir = "./logs"
if os.path.exists(logs_base_dir):
    shutil.rmtree(logs_base_dir)
os.makedirs(logs_base_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logs_base_dir, histogram_freq=1)
cce=tf.keras.losses.CategoricalCrossentropy()
kl=tf.keras.losses.KLDivergence()
mulClassifier.compile(optimizer=optimizer,loss=[cce,'mse'],loss_weights=[1.,0.5],metrics=['accuracy'])
checkpoint_filepath='/home/azka/My KL Grade Project/Notebooks/modelNormal'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_classifier_loss',
    mode='min',
    save_best_only=True)

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('mul_classifier.csv', separator=';')

In [ ]:
X_train012.shape

In [ ]:
classify_train = mulClassifier.fit([X_train012],[Y_train012,X_train012], batch_size=8,validation_data=([X_val012],[Y_val012,X_val012]),epochs=30,verbose=1,shuffle=True,callbacks=[csv_logger,model_checkpoint_callback])#,tensorboard_callback])


In [ ]:
import pandas as pd
mulClassifier.load_weights(checkpoint_filepath)
classify_train = pd.read_csv('mul_classifier.csv', sep=';')

In [ ]:
mulClassifier.evaluate([X_train012],[Y_train012,X_train012])

In [ ]:
mulClassifier.evaluate([X_val012],[Y_val012,X_val012])

In [ ]:
mulClassifier.evaluate([X_test012],[Y_test012,X_test012])

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
sns.set(font_scale=2)
pred=mulClassifier.predict(X_test012)
pred = pred[0].argmax(axis=1)
actual = Y_test012.argmax(axis=1)
accuracy = (pred == actual).sum()/len(pred)
cm=confusion_matrix(actual,pred)
sns.heatmap(cm, annot=True, fmt='g')

# Pipeline

In [ ]:
class KLGradeModel(keras.Model):
    def __init__(self, autoNet, binClassifier, mulClassifier, **kwargs):
        super().__init__(**kwargs)
        self.autoNet = autoNet
        self.binClassifier = binClassifier
        self.mulClassifier = mulClassifier
    def __call__(self, inputs):
        yNormalSevere = self.autoNet(inputs)
        y_s = tf.argmax(yNormalSevere[0],axis=1)==0
        normal_images = tf.boolean_mask(inputs, y_s)
        severe_images = tf.boolean_mask(inputs, ~y_s)
        Y_12 = self.mulClassifier(normal_images)
        Y_34= self.binClassifier(severe_images)
        return yNormalSevere,y_s,Y_12, Y_34

In [ ]:

model=KLGradeModel(autoNet,binClassifier,mulClassifier)
yNormalSevere,y_s,Y_12, Y_34=model(X_test)

In [ ]:
def Predictions(yNS,Y_12, Y_34):
    predictions=[]
    x=0
    y=0
    y34=np.array(tf.argmax(Y_34,axis=1))
    y12=np.array(tf.argmax(Y_12,axis=1))
    y_s=np.array(tf.argmax(yNS,axis=1))
    for s in y_s:
        if s==0:    
            if y12[x]==0:
                predictions.append(0)
            elif y12[x]==1:
                predictions.append(1)
                
            elif y12[x]==2:
                predictions.append(2)
            x=x+1
        if s==1:
            if y34[y]==0:
                predictions.append(3)
            elif y34[y]==1:
                predictions.append(4)

            y=y+1
    return np.array(predictions)

In [ ]:
preds=Predictions(yNormalSevere[0].numpy(),Y_12[0].numpy(), Y_34[0].numpy())

In [ ]:
Actual=Y_test2.argmax(axis=1)
#Actual=np.where(Actual==0,1,Actual)

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
sns.set(font_scale=2)
accuracy = (preds == Actual).sum()/len(preds)
cm=confusion_matrix(Actual,preds)
sns.heatmap(cm, annot=True, fmt='g')

In [ ]:
accuracy

#  Visualizations

In [ ]:
plt.imshow(X_train[0],'gray')

In [ ]:
X_train.shape

In [ ]:
plt.imshow(X_train012[1560].squeeze(),'gray')

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(15, 15))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    ax = fig.add_subplot(rows, columns, i)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.imshow(np.squeeze(X_train)[i], 'gray')
plt.show()

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(15, 15))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    ax = fig.add_subplot(rows, columns, i)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.imshow(np.squeeze(out_imgs)[i], 'gray')
plt.show()